In [48]:
import tensorflow as tf 
import numpy as np

- Tensorflow is a library for numerical computation where data is flows through the graph
- Data in Tensorflow is represented by n-dimensional arrays called Tensors 
- Graph is made of data(Tensors) and mathematical operators
    - Nodes on the graph: represents mathematical operations
    - Edges on the graph: represents Tensors that flow between operatioins 

# 1) Graph in Tensorflow

- Graph is the backbone of TensorFlow and every computation/operation/variables reside on the graph. 
- Everything that happens in the code, resides on a default graph provided by TensorFlow. 
- You can access this graph by

In [6]:
graph = tf.get_default_graph()
graph.get_operations() # print [] since no operations created yet 

[]

In [7]:
for op in graph.get_operations():
    print(op.name)

# 2)TensorFlow Session
- A graph is used to define operations, but the operations are only run within a session. 
- Graphs and sessions are created independently of each other. 
- You can imagine graph to be similar to a blueprint, and a session to be similar to a construction site.

- Graph only defines the computations or builds the blueprint. However, there are no variables, no values unless we run the graph or part of the graph within a session.

### You can create a session like this:

In [9]:
sess=tf.Session()
pass 
sess.close() # Whenever, you open a session, you need to remember to close it

#####  you can use ‘with block’ like this

In [19]:
f = tf.constant(4)
with tf.Session() as sess:
    sess.run(f)

# 3) Tensors in TensorFlow
- TF holds data in Tensors which are similar to numPy multi-dimensional arrays

### a) Constant
- Constants are whose value can't be changed

In [22]:
a = tf.constant(3.0)
a

<tf.Tensor 'Const_3:0' shape=() dtype=float32>

In [23]:
print(a)

Tensor("Const_3:0", shape=(), dtype=float32)


- Can access the constant "a" only when running it inside the session

In [25]:
with tf.Session() as sess:
    print(sess.run(a)) # accessing constant "a"

3.0


### b) Variables: 
- Can hold diffrent variables
- They need to be separately initialized by an init op

In [28]:
b = tf.Variable(2.0, name='Test_var')
b

<tf.Variable 'Test_var:0' shape=() dtype=float32_ref>

In [29]:
init_op = tf.global_variables_initializer()

- This will add init_op to our tensorflow default graph.
- Need to run init_op before you try to access your variable

In [31]:
with tf.Session() as sess:
    sess.run(init_op) # initializing variables
    print(sess.run(b)) # accessing variable "b" 

2.0


- Operations on the graph

In [32]:
graph = tf.get_default_graph()
for op in graph.get_operations():
    print(op.name)

Const
Const_1
Const_2
Const_3
Test_var/initial_value
Test_var
Test_var/Assign
Test_var/read
init


### c) Placeholders 
- They are tensors which are waiting to be initialized/fed.
- Used for training data which is only fed when the code is actually run inside a session
- feed_dict:
  - What is fed to Placeholder
  - keys value pairs for holding data

In [33]:
a = tf.placeholder("float")
b = tf.placeholder("float")
y = tf.multiply(a, b)
feed_dict = {a: 2, b: 3}
with tf.Session() as sess:
    print(sess.run(y, feed_dict))

6.0


# TensorFlow functions 

#### 1) Create a random normal distribution:
- Use random_normal to create random values from a normal distribution. 
- In this example, w is a variable which is of size 784*10 with random values with standard deviation 0.01

In [35]:
w = tf.Variable(tf.random_normal([784, 10], stddev=0.01))

#### 2) Reduce_mean:
- calculates the mean of an array

In [46]:
b = tf.Variable([10, 20, 30, 40, 50, 60], name='t')
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(tf.reduce_mean(b)))

35


#### 3) ArgMax:
- Very similar to python argmax. 
- Gets you the maximum value from a tensor along the specified axis

In [47]:
a=[ [0.1, 0.2, 0.3],
    [20,  2,   3  ]]
b = tf.Variable(a, name='b')
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(tf.argmax(b,1)))

[2 0]


# Lineer regression model with TensorFlow
- Problem statement: 
  - In linear regression, you get a lot of data-points and try to fit them on a straight line.
  - For this example, we will create 100 datapoints and try to fit them into a line
  
### a) Creating training data set:
- trainX has values between -1 and 1
- trainY has 3 times the trainX and some randomness

In [49]:
trainX = np.linspace(-1, 1, 101)
trainY = 3 * trainX + np.random.randn(*trainX.shape) * 0.33

### b) Placeholder

In [50]:
X = tf.placeholder('float')
Y = tf.placeholder('float')

### c) Modeling
- Linear regression model is y_model=w*x 
- we have to calculate the value of w through our model

In [53]:
# initialize weight to zero and create a model to find optimal weight
w = tf.Variable(0.0, name='wieghts')
y_model = tf.multiply(X, w)

# define the cost as square of (Y-y_model)
cost = (tf.pow(Y-y_model, 2))

# Use Gradient Descent Optimizer to update the gradient after each iteration while minimizing the specified cost
# With learning rate = 0.01
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

### d) Training 
- Till this point, we have only defined the graph. No computation has happened
- None of the TensorFlow variables have any value. 
- In order to run this graph, we need to create a Session and run. 
- Before that we need to create the init_op to initialize all variables

In [56]:
init = tf.global_variables_initializer()

In [57]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(100):
        for (x, y) in zip(trainX, trainY):
            sess.run(train_op, feed_dict={X: x, Y:y})
    print(sess.run(w))

3.0805


## Notes:
- #### new session block after this code and try to print w

In [59]:
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(w))

0.0


- Expected output is 0.0. 
- That’s the idea of symbolic computation. 
- Once, we have gotten out of session created earlier, all the operations cease to exist.